# **Human Action Recognition (HAR)**

In [4]:
# Set working environment variable (keep one true)
my_machine = False
colab = True
# Set TPU bool (can be used in colab)
# Current versioning error using a TPU so keep false (to be fixed)
tpu = False

# Assumed packages installed on local machine
# If colab install keras tuner (not included in colab)
if colab:
    %pip install -q -U keras-tuner

# Major imports
import os
import zipfile
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib as mp
import keras_tuner

# Partial imports
from tensorflow import keras
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout

# Confirm completion
print( "Everything successfully imported!" )

     |████████████████████████████████| 133 kB 32.6 MB/s 
Everything successfully imported!


In [ ]:
# If on your machine check for GPU and and enable memory growth
# Do not need to run if not using a GPU
if not tpu:
    physical_devices = tf.config.experimental.list_physical_devices('GPU')
    if not physical_devices:
        print( "No GPU recognized!" )
    else:
        print( "Number of GPUs recognized: ", len(physical_devices) )
        tf.config.experimental.set_memory_growth(physical_devices[0], True)

# If using Colab you can connect and init a TPU for training
if tpu and colab:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    tpu_stat = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
# Collect train and test data from csv files
# HAR data found on Kaggle
if my_machine:
    ROOT_DIR = os.getcwd()
    train_set_csv = pd.read_csv( os.path.join(ROOT_DIR, "Training_set.csv") )
    test_set_csv = pd.read_csv( os.path.join(ROOT_DIR, "Testing_set.csv") )

if colab:
    from google.colab import files
    upload = files.upload()
    train_set_csv = pd.read_csv('Training_set.csv')
    test_set_csv = pd.read_csv('Testing_set.csv')

num_classes = len(train_set_csv['label'].unique())
print( "Total number of action classes: ", num_classes )
print( "List of action classes: " )
print( train_set_csv['label'].unique() )

In [ ]:
# Get path to or upload train and test data
if my_machine:
    TRAIN_DIR = os.path.join(ROOT_DIR, "train")
    TEST_DIR = os.path.join(ROOT_DIR, "test")
if colab:
    from google.colab import files
    upload = files.upload()
    with zipfile.ZipFile("train.zip", "r") as zip_ref:
        zip_ref.extractall()
    TRAIN_DIR = "train"
    with zipfile.ZipFile("test.zip", "r") as zip_ref:
        zip_ref.extractall()
    TEST_DIR = "test"

# Create train dataset and apply transformations
train_image_generator = ImageDataGenerator(
    rescale=1./255, 
    horizontal_flip=True, 
    shear_range=0.2, 
    zoom_range=0.2
)
train_dataset = train_image_generator.flow_from_dataframe(
    dataframe=train_set_csv,
    directory=TRAIN_DIR,
    x_col='filename',
    y_col='label',
    target_size=(224, 224),
    class_mode='categorical'
)

# Create test dataset and apply transformations
test_image_generator = ImageDataGenerator(rescale=1./255)
test_dataset = test_image_generator.flow_from_dataframe(
    dataframe=test_set_csv, 
    directory=TEST_DIR,
    x_col='filename',
    y_col=None,
    target_size=(224, 224),
    class_mode=None
)

In [ ]:
def build_cnn_model():
    # Create basic CNN Model
    cnn = Sequential()
    cnn.add( Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[224, 224, 3]) )
    cnn.add( MaxPool2D(pool_size=2, strides=2) )
    cnn.add( Conv2D(filters=32, kernel_size=3, activation='relu') )
    cnn.add( MaxPool2D(pool_size=2, strides=2) )
    cnn.add( Flatten() )
    cnn.add( Dense(units=448, activation='relu') )
    cnn.add( Dense(units=num_classes, activation='sigmoid') )
    # Compile modile and return
    cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return cnn

# Check for tpu and then create model (avoids rewriting the model)
if not tpu:
    cnn = build_cnn_model()
if tpu and colab:
    with tpu_stat.scope():
        cnn = build_cnn_model()

In [ ]:
# Fit basic model and save weights
cnn.fit(x=train_dataset, validation_data=test_dataset, epochs=10)
cnn.save_weights("cnn_har.h5")

In [ ]:
# ------------------------------ WORK IN PROGRESS ------------------------------
def build_vgg_model(hp):
    # Create base cnn layers from VGG16 but cut off dense layers to use our own instead
    # Use imagenet weights and lock them to be untrainable
    vgg = Sequential()
    pretrained_model= tf.keras.applications.VGG16(include_top=False, input_shape=(224, 224, 3), pooling='max', weights='imagenet')
    for layer in pretrained_model.layers:
            layer.trainable=False
    # Add dense layers to VGG16 for our personal use with 15 classes output
    # Use keras tuner for number of internal dense layers and number of nodes for each dense layer
    vgg.add( pretrained_model )
    vgg.add( Flatten() )
    for i in range(hp.Int("num_layers", 1, 3)):
        vgg.add( Dense(hp.Int("nodes_{i}", min_value=32, max_value=512, step=32), activation='relu') )
    # Output layer
    vgg.add( Dense(15, activation='softmax') )
    # Compile model and return
    vgg.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return vgg

# Check for tpu and then create model (avoids rewriting the model)
if not tpu:
    vgg = build_vgg_model(keras_tuner.HyperParameters())
if tpu and colab:
    with tpu_stat.scope():
        vgg = build_vgg_model(keras_tuner.HyperParameters())

In [ ]:
# ------------------------------ WORK IN PROGRESS ------------------------------
# Define tuner search and check summary
tuner = keras_tuner.RandomSearch(
    hypermodel=build_vgg_model,
    objective="val_accuracy",
    overwrite=True,
    directory="/",
    project_name="vgg_har_model",
)
tuner.search_space_summary()

In [ ]:
# ------------------------------ WORK IN PROGRESS ------------------------------
# Start tuner search for optimal model
tuner.search(x=train_dataset, validation_data=test_dataset, epochs=3)

In [ ]:
# ------------------------------ WORK IN PROGRESS ------------------------------
# Fit model and save resulting weights
vgg.fit(x=train_dataset, validation_data=test_dataset, epochs=50)
vgg.save_weights("vgg_har.h5")
# If using Colab it will save weights to local machine
if colab:
    from google.colab import files
    files.download("vgg_har.h5")

In [ ]:
# Create labels dict for reference
labels_ref = (train_dataset.class_indices)
labels = {}
for name, index in labels_ref.items():
    labels[index] = name

# Print labels dictionary evenly for a visual
print( "------------------------------" )
print( "Labels: " )
for index in range(len(labels)):
    if index < 10:
        print( " " + str(index) + ": " + labels[index] )
    else:
        print( str(index) + ": " + labels[index] )

# Collect 5 random pictures from test files and run predictions
test_files = os.listdir(TEST_DIR)
random_numbers = np.random.randint(low=0, high=len(test_files), size=5)
for i in random_numbers:
    # Collect test file and handle image
    test_file = TEST_DIR + "/" + test_files[i]
    test_image = keras.preprocessing.image.load_img(test_file, target_size=(224, 224))
    test_image = keras.preprocessing.image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)
    test_file = mp.image.imread(test_file)
    plot = mp.pyplot.imshow(test_file)
    print( "------------------------------" )
    print( "\n" )
    print( test_files[i] )
    print( "\n" )
    mp.pyplot.show()
    print( "\n" )
    # Run predictions from tested model
    #prediction = cnn.predict([[test_image]])
    prediction = vgg.predict([[test_image]])
    results = {}
    for index in range(len(prediction[0])):
        results[index] = (prediction[0][index])
    sorted_results = sorted(results, key=results.get, reverse=True)
    for index in sorted_results:
        print( labels[index] + ": " + str(results[index]) )
        print( "\n" )